# Aufgabe 4: Preprocessing & Pipelines

Scikit-Learn ermöglicht es mittels <i>Pipelines</i>, verschiedene Vorverarbeitungsschritte (Normalisierung, Dimensionalitätsreduktion, etc.) mit einem Klassifier zu verbinden. In dieser Aufgabebeschäftigen wir uns mit dem Workflow von Datenverarbeitungsschritten mittels <i>Pipelines</i>.

a)  Laden Sie den <i>Breast Cancer Wisconsin dataset</i> unter:

https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data

Dieses Dataset beinhaltet ein <i>binary</i> Label für Krebsdiagnostic (M=malignant, B=benign) und mehrere numerische Features, berechnet aus digitalisierten Bildern von Zellkernen.

In [51]:
# import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split  # cross_validation is depricated
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV

In [52]:
data = pd.read_csv("./data/wdbc.data", 
                   names=["id","diagnosis","radius_mean","texture_mean","perimeter_mean",
                          "area_mean","smoothness_mean","compactness_mean","concavity_mean",
                          "concave_points_mean","symmetry_mean", "fractal_dimension_mean", 
                          "radius_se","texture_se","perimeter_se","area_se","smoothness_se",
                          "compactness_se","concavity_se", "concave_points_se","symmetry_se",
                          "fractal_dimension_se", "radius_worst","texture_worst","perimeter_worst",
                          "area_worst","smoothness_worst","compactness_worst","concavity_worst",
                          "concave_points_worst","symmetry_worst","fractal_dimension_worst"])
data.tail()  # contains both maligant and benign

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
568,92751,B,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


Die Datei wurde im Ordner <i>Data</i> gespeichert. Die Spaltennamen werden aus der Kaggle-Challenge übernommen. Die Datei enthält mit der ID und Diagnose 32 Spalten. Die Skalierung der Daten scheint je nach Spalte sehr unterschiedlich zu sein.
Quelle der Spaltennamen: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

b) Nutzen Sie die `sklearn.preprocessing.LabelEncoder` um die binary Label in ein numerisches Attribut zu kodieren.

In [53]:
le = LabelEncoder()
le.fit(data["diagnosis"])  # Fits the label encoder, so that it knows all possible values
data["diagnosis"] = le.transform(data["diagnosis"])
data.tail()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
564,926424,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
568,92751,0,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


Die Diagnose wurde in ein binäres Label umgewandelt. Bösartige ("maligant") Brustkrebse erhalten den Wert 1. Gutartige ("benign") Brustkrebse erhalten den Wert 0.

c) Mit `sklearn.cross_validation.train_test_split` splitten Sie die Daten in Trainings-(80%) und Testdaten (20%). Setzen Sie `random_state=1`.

In [54]:
X = data.drop(["id", "diagnosis"], axis=1)
y = data["diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print("X_train: %d, y_train: %d, X_test: %d, y_test: %d" % (len(X_train), len(y_train), len(X_test), len(y_test)))

X_train: 455, y_train: 455, X_test: 114, y_test: 114


Als erklärende Variablen werden alle Variablen außer der ID und der Diagnose festgelegt. Die ID ist kein Messwert, sondern wird ausschließlich zum auseinanderhalten von gleichen Zeilen verwendet. Die erklärende Variable ist die Diagnose. Der Trainingsdatensatz enthält 455 Elemente. Der Testdatensatz umfasst 114 Elemente.

d) Als Preprocessingschritte nutzen Sie `sklearn.preprocessing.StandardScaler`für feature scaling und für Dimensionalitätsreduktion ein PCA mit `sklearn.decomposition.PCA` und mit `n_components=2`. Als Klassifikator nutzen Sie `sklearn.linear_model.LogisticRegression` mit `random_state=1`. Packen Sie alle Schritte in eine Pipeline(`sklearn.pipeline.Pipeline`).

In [ ]:
scaler = StandardScaler()
pca = PCA(n_components=2)
clf = LogisticRegression(random_state=1, solver='lbfgs')
pipeline_pca = Pipeline([('StandardScaler', scaler), ('PCA', pca), ('LogisticRegression', clf)])
pipeline_pca.fit(X_train, y_train);

Zunächst werden die Objecte der einzelnen Komponenten mit den vorgegebenen Werten initialisiert. Bei der Logistischen Regression wird zusätzlich der Parameter `solver` auf 'lbfgs' gesetzt, um eine Warnung zu umgehen. Dann wird die Pipeline mit den Komponenten initialisiert und auf die Trainingsdaten angepasst.

e) Testen Sie mittels `pipeline.score`die Genauigkeit des Modells. Sie sollten eine Genauigkeit von 0.947 erreichen.

In [56]:
pipeline_pca.score(X_test, y_test)

0.9473684210526315

Die Genauigkeit entspricht der Vorgegebenen.

f) Statt PCA nutzen Sie ein Recursive Feature Elimination(RFE) zur Feature-Auswahl (`sklearn.feature_selection.RFECV`). Wie viele und welche Features sind für die Klassifikation interessant? Welche (max.) Genauigkeit auf die Testdaten kann mit diesem Schritt (statt PCA) erreicht werden?

In [57]:
scaler = StandardScaler()
clf = LogisticRegression(random_state=1, solver='lbfgs')
selector = RFECV(clf, step=1, cv=5) # remove 1 feature each iteration, use 5-fold cross-validation
pipeline_rfecv = Pipeline([('StandardScaler', scaler), ('RFECV', selector), ('LogisticRegression', clf)])
pipeline_rfecv.fit(X_train, y_train);

Analog zu der Pipeline in d) werden erst die einzelnen Komponenten initialisiert. Die PCA wurde durch die RFE ersetzt. In jedem Schritt wird ein Merkmal gelöscht und mit 5-Fold Crossvalidation überprüft, ob die Genauigkeit sich verbessert.

In [61]:
pipeline_rfecv.score(X_test, y_test)

0.9824561403508771

Die Pipeline schneidet durchschnittlich um ca. 3,5 Prozentpunkte besser ab, als die PCA-Pipeline.

In [59]:
interesting_feature_names = [feature_name for feature_name, feature_mask 
                             in zip(X.columns.values, selector.support_) if feature_mask]
print("Number of features: %d" % len(interesting_feature_names))
print("Features:\t%s" % "\n\t\t".join(interesting_feature_names))

Number of features: 22
Features:	radius_mean
		texture_mean
		perimeter_mean
		area_mean
		compactness_mean
		concavity_mean
		concave_points_mean
		radius_se
		perimeter_se
		area_se
		compactness_se
		concave_points_se
		fractal_dimension_se
		radius_worst
		texture_worst
		perimeter_worst
		area_worst
		smoothness_worst
		concavity_worst
		concave_points_worst
		symmetry_worst
		fractal_dimension_worst


22 der 30 ursprünglichen, erklärenden Variablen werden behalten.